In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
construction_df = pd.read_csv('../Street_Closures.csv')
print(construction_df)


                                               the_geom  SEGMENTID  \
0     MULTILINESTRING ((-74.12448960061522 40.631134...       9813   
1     MULTILINESTRING ((-74.19879341875411 40.522175...       1098   
2     MULTILINESTRING ((-74.19879341875411 40.522175...       1098   
3     MULTILINESTRING ((-73.95032596692067 40.657837...      28743   
4     MULTILINESTRING ((-73.95032596692067 40.657837...      28743   
...                                                 ...        ...   
1421  MULTILINESTRING ((-73.77137893497239 40.699014...      61101   
1422  MULTILINESTRING ((-73.77215742854926 40.711487...      61148   
1423  MULTILINESTRING ((-73.77215742854926 40.711487...      61148   
1424  MULTILINESTRING ((-73.76969922559675 40.697065...      61191   
1425  MULTILINESTRING ((-73.76969922559675 40.697065...      61191   

                     OFT      ONSTREETNAME   FROMSTREETNAME     TOSTREETNAME  \
0     524570523900544658    CORNELL AVENUE       CLOVE ROAD    RAYMOND PLACE   

In [5]:

construction_df.head()

,the_geom,SEGMENTID,OFT,ONSTREETNAME,FROMSTREETNAME,TOSTREETNAME,BOROUGH_CODE,WORK_START_DATE,WORK_END_DATE,UNIQUEID,PURPOSE
0,MULTILINESTRING ((-74.12448960061522 40.631134...,9813,524570523900544658,CORNELL AVENUE,CLOVE ROAD,RAYMOND PLACE,S,02/26/2025 12:00:00 AM,03/28/2025 12:00:00 AM,6820e945819f7e89ecf91829a95a6a3a8a0844b1621356...,REPAIR GAS
1,MULTILINESTRING ((-74.19879341875411 40.522175...,1098,570299525908547300,SINGLETON STREET,DEAD END,SEGUINE AVENUE,S,12/26/2024 12:00:00 AM,03/26/2025 12:00:00 AM,c4a5d52564922dfb3cd4e3c1feeb585df232e1c9feaa6e...,INSTALLATION OF POLES
2,MULTILINESTRING ((-74.19879341875411 40.522175...,1098,570299525908547300,SINGLETON STREET,DEAD END,SEGUINE AVENUE,S,12/26/2024 12:00:00 AM,03/25/2025 12:00:00 AM,f83d0277026f1d2b8c0617bc658e849360fc3f3c95168f...,INSTALLATION OF POLES
3,MULTILINESTRING ((-73.95032596692067 40.657837...,28743,348230364830367030,HAWTHORNE STREET,NEW YORK AVENUE,NOSTRAND AVENUE,B,02/24/2025 12:00:00 AM,03/22/2025 12:00:00 AM,8f45e12feccc75367e822dedc29d6d74c516ea0d2b2c37...,OCCUPANCY OF SIDEWALK AS STIPULATED
4,MULTILINESTRING ((-73.95032596692067 40.657837...,28743,348230364830367030,HAWTHORNE STREET,NEW YORK AVENUE,NOSTRAND AVENUE,B,02/24/2025 12:00:00 AM,03/22/2025 12:00:00 AM,512cb0dc3683a39a55fb776fdb9786543acedaf1c0af80...,OCCUPANCY OF ROADWAY AS STIPULATED


In [7]:
construction_df.tail()

,the_geom,SEGMENTID,OFT,ONSTREETNAME,FROMSTREETNAME,TOSTREETNAME,BOROUGH_CODE,WORK_START_DATE,WORK_END_DATE,UNIQUEID,PURPOSE
1421,MULTILINESTRING ((-73.77137893497239 40.699014...,61101,442940446350449520,DUNKIRK STREET,GALWAY AVENUE,HILBURN AVENUE,Q,03/10/2025 12:00:00 AM,05/01/2025 12:00:00 AM,d8bee4fa840e2d9e260e1d9df9d0f20f12b1aa3c039ab6...,DOT IN-HOUSE MILLING
1422,MULTILINESTRING ((-73.77215742854926 40.711487...,61148,418290425220425240,91 AVENUE,187 PLACE,188 STREET,Q,03/10/2025 12:00:00 AM,05/01/2025 12:00:00 AM,4ed5b29f15c55773a4bca9056242e9e0f985bf64e65aac...,DOT IN-HOUSE PAVING
1423,MULTILINESTRING ((-73.77215742854926 40.711487...,61148,418290425220425240,91 AVENUE,187 PLACE,188 STREET,Q,03/10/2025 12:00:00 AM,05/01/2025 12:00:00 AM,ca7bc5ecfcfdb0c987c0210ad38a4c6dbfb9d521fe18c9...,DOT IN-HOUSE MILLING
1424,MULTILINESTRING ((-73.76969922559675 40.697065...,61191,442940451650456350,DUNKIRK STREET,JORDAN AVENUE,MURDOCK AVENUE,Q,03/10/2025 12:00:00 AM,05/01/2025 12:00:00 AM,5e0dd125987aebde737382aa1ee0266d813a363fbaf8bf...,DOT IN-HOUSE MILLING
1425,MULTILINESTRING ((-73.76969922559675 40.697065...,61191,442940451650456350,DUNKIRK STREET,JORDAN AVENUE,MURDOCK AVENUE,Q,03/10/2025 12:00:00 AM,05/01/2025 12:00:00 AM,a6ab79f067e08f87e0c9d157e8953c8bafc74d91e3b723...,DOT IN-HOUSE PAVING


In [44]:
construction_df.shape

(1426, 11)

In [42]:
construction_df.BOROUGH_CODE

0       S
1       S
2       S
3       B
4       B
       ..
1421    Q
1422    Q
1423    Q
1424    Q
1425    Q
Name: BOROUGH_CODE, Length: 1426, dtype: object

In [45]:
construction_df.dtypes

the_geom           object
SEGMENTID           int64
OFT                 int64
ONSTREETNAME       object
FROMSTREETNAME     object
TOSTREETNAME       object
BOROUGH_CODE       object
WORK_START_DATE    object
WORK_END_DATE      object
UNIQUEID           object
PURPOSE            object
dtype: object

In [8]:
construction_df.columns

Index(['the_geom', 'SEGMENTID', 'OFT', 'ONSTREETNAME', 'FROMSTREETNAME',
       'TOSTREETNAME', 'BOROUGH_CODE', 'WORK_START_DATE', 'WORK_END_DATE',
       'UNIQUEID', 'PURPOSE'],
      dtype='object')

In [46]:
construction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426 entries, 0 to 1425
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   the_geom         1426 non-null   object
 1   SEGMENTID        1426 non-null   int64 
 2   OFT              1426 non-null   int64 
 3   ONSTREETNAME     1426 non-null   object
 4   FROMSTREETNAME   1426 non-null   object
 5   TOSTREETNAME     1426 non-null   object
 6   BOROUGH_CODE     1426 non-null   object
 7   WORK_START_DATE  1426 non-null   object
 8   WORK_END_DATE    1426 non-null   object
 9   UNIQUEID         1426 non-null   object
 10  PURPOSE          1426 non-null   object
dtypes: int64(2), object(9)
memory usage: 122.7+ KB


In [47]:
construction_df.WORK_END_DATE == construction_df.WORK_START_DATE

0       False
1       False
2       False
3       False
4       False
        ...  
1421    False
1422    False
1423    False
1424    False
1425    False
Length: 1426, dtype: bool

In [48]:
print(construction_df.head(5))

                                            the_geom  SEGMENTID  \
0  MULTILINESTRING ((-74.12448960061522 40.631134...       9813   
1  MULTILINESTRING ((-74.19879341875411 40.522175...       1098   
2  MULTILINESTRING ((-74.19879341875411 40.522175...       1098   
3  MULTILINESTRING ((-73.95032596692067 40.657837...      28743   
4  MULTILINESTRING ((-73.95032596692067 40.657837...      28743   

                  OFT      ONSTREETNAME   FROMSTREETNAME     TOSTREETNAME  \
0  524570523900544658    CORNELL AVENUE       CLOVE ROAD    RAYMOND PLACE   
1  570299525908547300  SINGLETON STREET         DEAD END   SEGUINE AVENUE   
2  570299525908547300  SINGLETON STREET         DEAD END   SEGUINE AVENUE   
3  348230364830367030  HAWTHORNE STREET  NEW YORK AVENUE  NOSTRAND AVENUE   
4  348230364830367030  HAWTHORNE STREET  NEW YORK AVENUE  NOSTRAND AVENUE   

  BOROUGH_CODE         WORK_START_DATE           WORK_END_DATE  \
0            S  02/26/2025 12:00:00 AM  03/28/2025 12:00:00 AM   
1 

In [50]:
print(construction_df[['FROMSTREETNAME', 'PURPOSE']].describe())

       FROMSTREETNAME                             PURPOSE
count            1426                                1426
unique            308                                  46
top              BEND  OCCUPANCY OF ROADWAY AS STIPULATED
freq               77                                 333


In [51]:
print(construction_df['BOROUGH_CODE'].unique())

['S' 'B' 'M' 'Q' 'X']


In [52]:
grouped_products = construction_df.groupby("BOROUGH_CODE")
units_amt = grouped_products["WORK_END_DATE"].max()
print(units_amt)


BOROUGH_CODE
B    06/17/2025 12:00:00 AM
M    05/27/2025 12:00:00 AM
Q    06/04/2025 12:00:00 AM
S    05/26/2025 12:00:00 AM
X    05/31/2025 12:00:00 AM
Name: WORK_END_DATE, dtype: object


In [54]:
grouped_products = construction_df.groupby("WORK_START_DATE")
units_amt = grouped_products["WORK_END_DATE"].max()
print(units_amt)

WORK_START_DATE
01/01/2025 12:00:00 AM    04/01/2025 12:00:00 AM
01/02/2025 12:00:00 AM    04/01/2025 12:00:00 AM
01/03/2025 12:00:00 AM    04/01/2025 12:00:00 AM
01/06/2025 12:00:00 AM    03/08/2025 12:00:00 AM
01/07/2025 12:00:00 AM    04/07/2025 12:00:00 AM
                                   ...          
12/23/2024 12:00:00 AM    03/23/2025 12:00:00 AM
12/26/2024 12:00:00 AM    03/26/2025 12:00:00 AM
12/28/2024 12:00:00 AM    03/28/2025 12:00:00 AM
12/30/2024 12:00:00 AM    03/30/2025 12:00:00 AM
12/31/2024 12:00:00 AM    03/10/2025 12:00:00 AM
Name: WORK_END_DATE, Length: 77, dtype: object
